<a href="https://colab.research.google.com/github/isdeniz/results-0207/blob/main/msc_bug_enhancement_question_llama3_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.0 MB/s eta 0:00:00


In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3")

In [4]:
pip install tqdm

In [5]:
import pandas as pd
from tqdm import tqdm

In [6]:
test = pd.read_csv('data/test_bug_enhancement_question_nlbse22_preprocessing_200_detailed_data.csv')
train = pd.read_csv('data/train_bug_enhancement_question_nlbse22_preprocessing_200_detailed_data.csv')
print('Number of issues in test dataset: ', test.shape, 'Number of issues in train dataset: ', train.shape)

Number of issues in test dataset:  (10537, 7) Number of issues in train dataset:  (94830, 7)


In [7]:
msc_dataset = pd.concat([train, test], ignore_index = True)
msc_dataset.reset_index(drop = True, inplace = True)
msc_dataset.shape[0]

105367

In [8]:
msc_dataset.head()

,original_issue_title,original_issue_body,preprocessed_title,preprocessed_body,text,labels_name,mapLabel
0,Issue with drillspeed for Laserdrills,**Platform**: *Windows 7*\r\n\r\n**Build**: *s...,issue with drillspeed for laserdrills,platform windows 7 build steam build 104.10 is...,time 2020-06-01T16:01:49Z author NONE repo Anu...,['bug'],bug
1,Anti-virus program was blocking game,**Platform**: Windows 10\r\n\r\n**Build**: 104...,anti virus program was blocking game,platform windows 10 build 104.10 issue see scr...,time 2020-06-03T11:25:20Z author CONTRIBUTOR r...,['bug'],bug
2,Game crashes with a nonexistent item name in t...,**Platform**: *Android/iOS/Mac/Windows/Linux*\...,game crashes with a nonexistent item name in t...,platform android ios mac windows linux linux b...,time 2020-06-03T14:27:30Z author CONTRIBUTOR r...,['bug'],bug
3,Graphical bug with legged units,**Platform**: Windows\r\n\r\n**Build**: Latest...,graphical bug with legged units,platform windows build latest bleeding edge is...,time 2020-06-03T17:50:49Z author NONE repo Anu...,['bug'],bug
4,Multiplayer incompatibilities due to mismatche...,**Platform**: *Android/iOS/Steam*\r\n\r\n**Bui...,multiplayer incompatibilities due to mismatche...,platform android ios steam build android 104.6...,time 2020-06-06T11:06:07Z author NONE repo Anu...,['bug'],bug


In [9]:
s = msc_dataset.shape[0] // 1000
print(s)

105


In [10]:
msc_dataset.to_pickle('data/msc_dataset.pkl')

In [15]:
i = 0
x = 1

while x < (s+1):
    i = (x - 1) * 1000
    n = x * 1000
    with tqdm(total = (n - i), ncols = 100) as pbar:
        while i < n:
            try:
                msc_dataset.loc[i,'llama3_prediction'] = llm.invoke(f"Considering the following issue where the issue title: {msc_dataset.loc[i,'original_issue_title']} and issue body: {msc_dataset.loc[i,'original_issue_body']}, please categorize this issue as either (i) bug report, or (ii) enhancement request, or (iii) something else. Your response should be either (i) bug report, or (ii) enhancement request, or (iii) something else. Do not provide any explanation.")
                pbar.update(1)
                i = i + 1
            except:
                print('exception')
                continue

    msc_dataset.to_pickle('data/llama3_8b_msc_bug_enhancement_question_predictions.pkl')
    print(f'Categorization of {n} issues are completed and saved')
    x = x + 1

100%|███████████████████████████████████████████████████████████| 1000/1000 [04:44<00:00,  3.51it/s]


Categorization of 1000 issues are completed and saved


100%|███████████████████████████████████████████████████████████| 1000/1000 [05:04<00:00,  3.29it/s]


Categorization of 2000 issues are completed and saved


100%|███████████████████████████████████████████████████████████| 1000/1000 [05:07<00:00,  3.25it/s]


...

Categorization of 103000 issues are completed and saved


100%|███████████████████████████████████████████████████████████| 1000/1000 [05:56<00:00,  2.81it/s]


Categorization of 104000 issues are completed and saved


100%|███████████████████████████████████████████████████████████| 1000/1000 [05:57<00:00,  2.80it/s]


Categorization of 105000 issues are completed and saved


In [16]:
print(x)

106


In [17]:
i = (x - 1) * 1000
n = msc_dataset.shape[0]
with tqdm(total = (n - i), ncols = 100) as pbar:
  while i < n:
    try:
      msc_dataset.loc[i,'llama3_prediction'] = llm.invoke(f"Considering the following issue where the issue title: {msc_dataset.loc[i,'original_issue_title']} and issue body: {msc_dataset.loc[i,'original_issue_body']}, please categorize this issue as either (i) bug report, or (ii) enhancement request, or (iii) something else. Your response should be either (i) bug report, or (ii) enhancement request, or (iii) something else. Do not provide any explanation.")
      pbar.update(1)
      i = i + 1
    except:
      print('exception')
      continue

msc_dataset.to_pickle('data/llama3_8b_msc_bug_enhancement_question_predictions.pkl')
print(f'Categorization of {n} issues are completed and saved')

100%|█████████████████████████████████████████████████████████████| 367/367 [02:14<00:00,  2.73it/s]


Categorization of 105367 issues are completed and saved
